In [1]:
import pandas as pd
import pickle
import numpy as np

In [2]:
import pickle

def dump_pickle(file_path, obj):
    with open(file_path, "wb") as dump_file:
        pickle.dump(obj, dump_file)

def load_pickle(file_path):
    with open(file_path, 'rb') as load_file:
        return pickle.load(load_file)

## 1 Анализ данных

Раскроем данные так, чтобы они приняли формат player -> ответ на вопрос
- отфильтруем турниры до 2019-01-01
- разобъем их на трейн тест
- отфильтруем турниры с пустыми полями 'mask', 'teamMembers', 'team'
- для каждого игрока создадим свою строку в данных
- для каждого вопроса создадим свою строку в даннных 

In [3]:
df_tournaments = pd.DataFrame(pd.read_pickle('tournaments.pkl')).transpose()
#отфильтруем по дате
tournaments_ids_all = set(df_tournaments[df_tournaments.dateStart >= '2019-01-01']['id'])
tournaments_ids_test = set(df_tournaments[df_tournaments.dateStart >= '2020-01-01']['id'])
tournaments_ids_train = tournaments_ids_all.difference(tournaments_ids_test)
len(tournaments_ids_all), len(tournaments_ids_train), len(tournaments_ids_test)

(1109, 687, 422)

In [4]:
def filter_df(tournaments_ids):
    df_results = pd.read_pickle('results.pkl')
    print("full df: ", len(df_results))
    results_all = {}
    for key, value in df_results.items():
        # берем турниры из tournaments_ids без пустых значений
        if key in tournaments_ids and len(value) > 0:
            valid = True
            # фильтруем турниры с пустой инфой по команде
            for team_data in value:
                if 'team' not in team_data or 'mask' not in team_data or 'teamMembers' not in team_data:
                    valid = False
                    continue
                if team_data['mask'] is None or team_data['team'] is None or team_data['teamMembers'] is None:
                    valid = False
                    continue
            if valid:
                results_all[key] = value
    print("filtered df: ", len(results_all))
    return results_all

df_train = filter_df(tournaments_ids_train)
df_test = filter_df(tournaments_ids_test)
dump_pickle('test.pkl', df_test)

def unpivot_players(df):
    df_results_cleaned = []
    for key, value in df.items():
        # чистим маску и переводим игроков из массива в отдельные строки
        for team_data in value:
            team = team_data['team']
            mask = str(team_data['mask']).replace('X', '0').replace('?', '0')
            players = team_data['teamMembers']
            for player in players:
                df_results_cleaned.append([key, team['id'], player['player']['id'], mask])
    df = pd.DataFrame(df_results_cleaned)
    df.columns = ['tournament_id', 'team_id', 'player_id', 'mask']
    return df

df_train = unpivot_players(df_train)

def unpivot_questions(df):
    df_results_cleaned = []
    for _, row in df.iterrows():
        tournament_id = row['tournament_id']
        team_id = row['team_id']
        player_id = row['player_id']
        mask = row['mask']
        # переводим маску из массива в отдельные строки
        for idx in range(len(mask)):
            df_results_cleaned.append([tournament_id, team_id, player_id, f"{tournament_id}_{idx}", mask[idx]])
    df = pd.DataFrame(df_results_cleaned)
    df.columns = ['tournament_id', 'team_id', 'player_id', 'question_id', 'target']
    return df

df_train = unpivot_questions(df_train)
compression_opts = dict(method='zip', archive_name='train.csv')
df_train.to_csv('train.zip', index=False, compression=compression_opts)

full df:  5528
filtered df:  671
full df:  5528
filtered df:  169


## 2 Baseline model

Построим модель на one-hot векторах из игроков и вопросов

Т.к. дальше в EM алгоритме нам нужно будет обучать модель на вероятностях, то построим модель по схеме предложенной тут:
https://stackoverflow.com/questions/42800769/scikit-learn-classification-on-soft-labels/60969923#60969923


In [5]:
df_train = pd.read_csv('train.zip', 
                       dtype={'tournament_id':np.int16, 'team_id':np.int32,
                              'player_id':np.int32, 'question_id':object, 'target':np.int8})

In [6]:
df_train.head()

,tournament_id,team_id,player_id,question_id,target
0,4772,45556,6212,4772_0,1
1,4772,45556,6212,4772_1,1
2,4772,45556,6212,4772_2,1
3,4772,45556,6212,4772_3,1
4,4772,45556,6212,4772_4,1


In [7]:
from sklearn.preprocessing import OneHotEncoder, Normalizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression

In [8]:
def sigmoid(x):
    ex = np.exp(x)
    return ex / (1 + ex)

In [9]:
feature_generation = ColumnTransformer(
    transformers=[
        ('OneHot', OneHotEncoder(), ['player_id', 'question_id'])
    ],
    remainder='drop',
    sparse_threshold=1
)

soft_pipe = Pipeline(
    steps=[
        ('feature_generation', feature_generation),
        #('scaler', StandardScaler()),
        #('classifier', LogisticRegression(solver='liblinear', max_iter=100))
        ('classifier', LinearRegression())
    ]
)

In [10]:
df_train.shape

(20968351, 5)

In [11]:
%%time
# logistic regression on soft labels
# https://stackoverflow.com/questions/42800769/scikit-learn-classification-on-soft-labels/60969923#60969923

y = np.clip(df_train['target'], 1e-8, 1 - 1e-8)   # numerical stability
inv_sig_y = np.log(y / (1 - y))  # transform to log-odds-ratio space

soft_pipe.fit(df_train, inv_sig_y)

CPU times: user 11min 14s, sys: 11min 6s, total: 22min 20s
Wall time: 4min 24s


Pipeline(steps=[('feature_generation',
                 ColumnTransformer(sparse_threshold=1,
                                   transformers=[('OneHot', OneHotEncoder(),
                                                  ['player_id',
                                                   'question_id'])])),
                ('classifier', LinearRegression())])

## 3 Качество рейтинг-системы

In [12]:
import dill

def dump_dill(file_path, obj):
    with open(file_path, "wb") as dump_file:
        dill.dump(obj, dump_file)
        
def load_dill(file_path):
    with open(file_path, "rb") as dump_file:
        return dill.load(dump_file)
    
dump_dill('logreg_model.dll', soft_pipe)

In [13]:
soft_pipe = load_dill('logreg_model.dll')

In [14]:
df_test = load_pickle('test.pkl')

In [15]:
# добудем веса из модели для каждого игрока
player_to_weight = {int(name.split('_')[-1]): rank for name, rank in zip(
    soft_pipe['feature_generation'].get_feature_names(),
    soft_pipe['classifier'].coef_) if name.split('_')[2] == 'x0'}

Посмотрим насколько совпадают топ 100 и наш рейтинг

Ссылка на официальный рейтинг: https://rating.chgk.info/players.php

In [16]:
official_top_100_ids = pd.read_csv('player_off_top.csv')[:100]
official_top_100_ids = set(official_top_100_ids[' ИД'])

player_to_weight_sorted = sorted(player_to_weight.items(), key=lambda kv: kv[1], reverse=True)
predicted_top_100_ids = set(k for k, v in player_to_weight_sorted[:100])

intersec = len(official_top_100_ids.intersection(predicted_top_100_ids))
print(f"Получается, что в наш топ 100 и в официальный топ 100 входит {intersec} человек")
print('Конечно, такая простая проверка не учитывает порядок ранжирования')

Получается, что в наш топ 100 и в официальный топ 100 входит 47 человек
Конечно, такая простая проверка не учитывает порядок ранжирования


In [17]:
from scipy import stats

Будем считать рейтинг команды как средний рейтинг ее игроков.

Если игрока нет в тестовых данных, то ставим ему в соответствие средний вес по всем игрокам в трейне

In [18]:
def get_real_rank(tournament):
    return [team['position'] for team in tournament]

def get_predicted_rank(tournament):
    avg_weight = np.mean(list(player_to_weight.values()))
    team_rating = []
    for idx, team in enumerate(tournament):
        weight = 0
        cnt = 0
        for player_info in team['teamMembers']:
            p_id = player_info['player']['id']
            try:
                weight += player_to_weight[p_id]
            except:
                weight += avg_weight
            cnt += 1
        try:
            mean = weight/cnt
        except:
            mean = 0
        team_rating.append((idx + 1, mean))
    team_rating = sorted(team_rating, key=lambda kv: kv[1], reverse=True)
    return [pos for pos, weight in team_rating]

In [19]:
def get_score(df, corr):
    x = [corr(get_real_rank(tour), get_predicted_rank(tour)).correlation for tour in df.values()]
    return np.nanmean(x)

for corr in [('Spearman', stats.spearmanr), ('Kendall ', stats.kendalltau)]:
    print(f"Average {corr[0]} correlation for df_test: {get_score(df_test, corr[1]):.3f}")

Average Spearman correlation for df_test: 0.776
Average Kendall  correlation for df_test: 0.622


## 4 EM схема

Скрытыми переменными $z_{i,q}$ являются ответил ли $i$-ый игрок в команде на вопрос $q$. Пусть $y_{tq}$ - ответ команды $T$ на вопрос $q$. Если команда не ответила на вопрос, то ни один игрок команды не ответил на вопрос, если ответила - то значит хотя бы один игрок ответил.

Величины $z_{i,q}$ будем моделировать логистическими регрессиями от *player_id* и *question_id*.

$$p\left(z_{i,q} \vert \overrightarrow{x_i}\right) \sim \sigma\left(\overrightarrow{x_i}\right)$$

Тогда *Expectation* шаг будет рассчитываться по формуле

$$ 
\mathbb{E} \left[ z^{(m+1)}_{i,q} \right] = 
\begin{cases}
0, \text{если } y_{tq} = 0,\\
p\left( z^{(m)}_{i,q} = 1 \vert \exists j \in t : z^{(m)}_{j,q} = 1\right), \text{ если } y_{tq} = 1. 
\end{cases} 
$$

Где
$$
p\left( z^{(m)}_{i,q} = 1 | \exists j \in t : z^{(m)}_{j,q} = 1\right) = \frac{\sigma (\overrightarrow{x^{(m)}_i})}{1-\prod_{k \in T} (1 - \sigma(\overrightarrow{x^{(m)}_k}))}
$$

На *Maximization* шаге будем бучать модель

$$\mathbb{E} \left[ z^{(m+1)}_{i,q} \right] \sim \sigma\left(\overrightarrow{x^{(m+1)}_i}\right)$$

В качестве начальных параметров возьмем параметры модели из бейзлайна

In [20]:
from functools import reduce

def pred_by_team(arr):
    arr = 1-arr
    return 1 - np.prod(arr)

In [21]:
df_train['new_target'] = df_train['target']
number_iter = 10

print('Baseline:')
for corr in [('Spearman', stats.spearmanr), ('Kendall ', stats.kendalltau)]:
    print(f"Average {corr[0]} correlation for df_test: {get_score(df_test, corr[1]):.3f}")

for i in range(number_iter):
    print(f"Iteration number: {i}")
    #if i == 0:
    #    df_train['pred'] = pipe.predict_proba(df_train)[:, 1]
    #if i > 0:
    
    # EXPECTATION STEP
    df_train['pred'] = sigmoid(soft_pipe.predict(df_train))
        
    pred_by_group = (df_train[df_train.target == 1]
             .groupby(['team_id', 'question_id'], as_index=False)['pred']
             .apply(pred_by_team)
             .rename(columns={'pred':'pred_by_team'}))
    df_train = pd.merge(df_train, pred_by_group, how='left', on=['team_id', 'question_id'])
    df_train['new_target'] = df_train['pred']/df_train['pred_by_team']
    df_train.loc[df_train['target'] == 0, 'new_target'] = 0

    df_train = df_train.drop(columns=['pred', 'pred_by_team'])

    # MAXIMIZATION STEP
    y = np.clip(df_train['new_target'], 1e-8, 1 - 1e-8)   # numerical stability
    inv_sig_y = np.log(y / (1 - y))  # transform to log-odds-ratio space
    
    soft_pipe.fit(df_train, inv_sig_y)
    
    # VALUATION STEP
    # добудем веса из модели для каждого игрока
    player_to_weight = {int(name.split('_')[-1]): rank for name, rank in zip(
        soft_pipe['feature_generation'].get_feature_names(),
        soft_pipe['classifier'].coef_) if name.split('_')[2] == 'x0'}
    
    for corr in [('Spearman', stats.spearmanr), ('Kendall ', stats.kendalltau)]:
        print(f"Average {corr[0]} correlation for df_test: {get_score(df_test, corr[1]):.3f}")

Baseline:
Average Spearman correlation for df_test: 0.776
Average Kendall  correlation for df_test: 0.622
Iteration number: 0


/home/alexander/miniconda3/envs/2_ml/lib/python3.7/site-packages/sklearn/compose/_column_transformer.py:440: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  FutureWarning)


Average Spearman correlation for df_test: 0.771
Average Kendall  correlation for df_test: 0.617
Iteration number: 1
Average Spearman correlation for df_test: 0.771
Average Kendall  correlation for df_test: 0.617
Iteration number: 2
Average Spearman correlation for df_test: 0.776
Average Kendall  correlation for df_test: 0.621
Iteration number: 3
Average Spearman correlation for df_test: 0.777
Average Kendall  correlation for df_test: 0.621
Iteration number: 4
Average Spearman correlation for df_test: 0.777
Average Kendall  correlation for df_test: 0.621
Iteration number: 5
Average Spearman correlation for df_test: 0.777
Average Kendall  correlation for df_test: 0.620
Iteration number: 6
Average Spearman correlation for df_test: 0.777
Average Kendall  correlation for df_test: 0.622
Iteration number: 7
Average Spearman correlation for df_test: 0.778
Average Kendall  correlation for df_test: 0.623
Iteration number: 8
Average Spearman correlation for df_test: 0.776
Average Kendall  correla

К сожалению, EM алгоритм не дал серьезного прироста, возможно стоило реализовывать честную лог-регрессию на pyTorch

## 5 Рейтинг лист турниров по сложности вопросов

По аналогии с игроками, достанем из модели сложность каждого вопроса (коэффициент модели для этого вопроса).

Сложность вопроса должна уменьшать вероятность правильного ответа, поэтому чем меньше этот коэффициент - тем сложнее вопрос.

Для каждого турнира возьмем среднее от сложности его вопросов. Такую величину будем считать сложностью турнира

In [22]:
question_to_weight = {name.split('_')[-2] + '_' + name.split('_')[-1]: rank for name, rank in zip(
    soft_pipe['feature_generation'].get_feature_names(),
    soft_pipe['classifier'].coef_) if name.split('_')[2] == 'x1'}

In [23]:
def mean_question_weight(arr):
    '''
    map questions to weights and take mean
    '''
    return np.mean(list(map(question_to_weight.get, arr)))

tournament_to_weight = df_train.groupby('tournament_id', as_index=False)['question_id'] \
    .apply(mean_question_weight) \
    .sort_values(by=['question_id']) \
    .rename(columns={"question_id": "questions_mean_difficulty"})

В топ 15 сложных турниров входят Чемпионаты и Первые лиги, что кажется разумным

In [25]:
top_n = 15
print(f"Топ {top_n} самых сложных турниров")
tournament_to_weight.head(top_n).merge(
    df_tournaments[["id", "name"]].set_index("id"),
    left_on="tournament_id", right_on="id",
)

Топ 15 самых сложных турниров


,tournament_id,questions_mean_difficulty,name
0,6149,-9.660520,Чемпионат Санкт-Петербурга. Первая лига
1,5928,-7.055213,Угрюмый Ёрш
2,5684,-6.259954,Синхрон высшей лиги Москвы
3,6101,-5.997683,Воображаемый музей
4,5159,-5.960070,Первенство правого полушария
5,5942,-5.665289,Чемпионат Мира. Этап 2. Группа В
6,5693,-5.602871,Знание – Сила VI
7,5465,-5.579934,Чемпионат России
8,5083,-5.482918,Ускользающая сова
9,5587,-5.452834,Записки охотника


В топ 15 простых турниров входят школьные и студенческие игры, а также игры lite

In [26]:
top_n = 15
print(f"Топ {top_n} самых простых турниров")
tournament_to_weight.tail(top_n).merge(
    df_tournaments[["id", "name"]].set_index("id"),
    left_on="tournament_id", right_on="id",
)

Топ 15 самых простых турниров


,tournament_id,questions_mean_difficulty,name
0,5009,6.470768,(а)Синхрон-lite. Лига старта. Эпизод III
1,5601,6.501741,Межфакультетский кубок МГУ. Отбор №4
2,6254,6.572293,Школьная лига
3,5698,6.592132,(а)Синхрон-lite. Лига старта. Эпизод VII
4,5702,6.610661,(а)Синхрон-lite. Лига старта. Эпизод IX
5,5955,6.775553,Школьная лига. III тур.
6,5936,6.830198,Школьная лига. I тур.
7,5012,6.887684,Школьный Синхрон-lite. Выпуск 2.5
8,5013,7.351077,(а)Синхрон-lite. Лига старта. Эпизод V
9,6003,7.484514,Второй тематический турнир имени Джоуи Триббиани
